Prediction generator for the new HuggingFace training pipeline

Written by Maksym and Artem

Last version update by Artem on Aug 20th  *(edit this line if you change anything)*

# Setup

Setting up libraries, importing local files, initalizing helper models

In [ ]:
# installing the necessary packages
!pip install transformers &> /dev/null
!pip install datasets &> /dev/null
!pip install pymorphy2 &> /dev/null
!pip install pymorphy2-dicts-uk &> /dev/null
!pip install spacy_udpipe &> /dev/null

In [ ]:
# all the imports we will need
import sys
import pymorphy2
import spacy_udpipe

# local imports
sys.path.append('/content/drive/MyDrive/STABLE/helpers')
from SpaceHandler import SpaceHandler
from Inflector import Inflector
from GrammarInterpreter import GrammarInterpreter
from PunctInterpreter import PunctInterpreter
from AntiTokenizer import AntiTokenizer
from FormExtractorInflector import FormExtractorInflector
from SurzhInterpreter import SurzhInterpreter

In [ ]:
# initializing the helper models
# spacy
spacy_path = '/content/drive/MyDrive/maksym-bondarenko/spelling/udpipe/ukrainian-iu-ud-2.5-191206.udpipe'

SPACY_UDPIPE_MODEL = spacy_udpipe.load_from_path(
    lang="uk",
    path=spacy_path,
)

# pymorphy 
morph = pymorphy2.MorphAnalyzer(lang='uk')

# locally imported classes
space_handler = SpaceHandler()
punct_interpreter = PunctInterpreter(space_handler=space_handler)
inflector = Inflector(morph)
grammar_interpreter = GrammarInterpreter(space_handler=space_handler,
                                         inflector=inflector)
surzh_interpreter = SurzhInterpreter(inflector, SPACY_UDPIPE_MODEL)

## Model Initialization

Initializing the specific models: punctuation, grammar

Surzhik does not need initialization (as for 8/20)

In [ ]:
# punctuation
model_name = 'bavovna-BNR'
checkpoint = 'checkpoint-25934'
model_type = 'punct'

punct_anti_tokenizer = AntiTokenizer(model_name=model_name, 
                                     checkpoint=checkpoint,
                                     model_type=model_type,
                                     space_handler = space_handler)

In [ ]:
# grammar
model_name = 'chornobaivka-25'
checkpoint = 'checkpoint-51588'
model_type = 'grammar'

grammar_anti_tokenizer = AntiTokenizer(model_name=model_name, 
                                       checkpoint=checkpoint,
                                       model_type=model_type,
                                       space_handler=space_handler)

## Playground

Predicting the tags for the sentences.

Don't forget to specify the interpreter types!

In [ ]:
sent = 'Без праці не витягну рибку з озерця'
words, tags = grammar_anti_tokenizer(sent)
out = grammar_interpreter(words, tags)
print(words)
print(tags)
print(out)

['$START', 'Без', 'праці', 'не', 'витягну', 'рибку', 'з', 'озерця']
['$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP']
Без праці не витягну рибку з озерця


In [ ]:
sent = 'Що за хуйня" сказав він'
words, tags = punct_anti_tokenizer(sent)
out = punct_interpreter(words, tags)
print(out)

Що за хуйня "сказав він."


In [ ]:
sent = "я виключив світло"
out = surzh_interpreter.antisurzhifier(sent)
print(out)

я вимкнув світло


# Final Brute

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/ukramarly-main/spelling')
import bruteforce
import spelling

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
аакуватий


In [ ]:
print(spelling.Spelling("привіт, красивйй юнчче!").spelling())

[['привіт'], [','], ['красивий', 'красивій'], ['юначе'], ['!']]


In [ ]:
print(bruteforce.Bruteforce("Самому красивому юнаку").samiy())
print(bruteforce.Bruteforce("я приїхав в Варшаву").v_u_i_y())
print(bruteforce.Bruteforce("я з Львова").z_zi_iz_zo())
print(bruteforce.Bruteforce("робити найбільш краще").samiy())
print(bruteforce.Bruteforce("вона є нерозумна").ne_adj())
print(bruteforce.Bruteforce("це є негарно").ne_adv())
print(bruteforce.Bruteforce("пів-сон хочу випити пів-відра півня Серед пів'Європи північної півкулі пів'яблука кримського півострова півроку").piv_mistake())

Найкрасивішому юнаку
я приїхав у Варшаву
я зі Львова
робити найкраще
вона є не розумна
це є не гарно
пів сон хочу випити пів відра півня Серед пів Європи північної півкулі пів яблука кримського півострова пів року
